# Task 3: Data Persistence. Implement data persistence using a lightweight database system (e.g., SQLite or JSON file). 
When a new fund is created or the performance of an existing fund is updated, the data should be stored persistently. For this task, SQLite3 library will be used given the ability to handle complex queries and organized data management

For the answer, I have prepared two method of requesting input from user in which the first one would be a good back-end function with no user GUI and the second comes with a simple Tkinter GUI for stand alone app development.

For first answer, a .py file has been also created and can be run directly in the any terminal or prompt.

Answer:

### Answer 1: No GUI

In [4]:
import sqlite3
import os

# Database file name
DB_FILE = 'funds.db'

# Establish connection to the database
connection = sqlite3.connect(DB_FILE)
cursor = connection.cursor()

# Ensure the database schema is created
# Create the funds table if it doesn't already exist
def initialize_database():
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS funds (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        fund_name TEXT NOT NULL UNIQUE,
        performance REAL NOT NULL
    )
    """)
    connection.commit()

# Add a new fund to the SQLite database
def create_fund(fund_name, performance):
    try:
        cursor.execute("INSERT INTO funds (fund_name, performance) VALUES (?, ?)", (fund_name, performance))
        connection.commit()
        print(f"Fund '{fund_name}' added successfully.")
    except sqlite3.IntegrityError:
        print(f"Error: A fund with the name '{fund_name}' already exists.")

# Update the performance of an existing fund
def update_fund(fund_name, new_performance):
    cursor.execute("UPDATE funds SET performance = ? WHERE fund_name = ?", (new_performance, fund_name))
    if cursor.rowcount > 0:
        connection.commit()
        print(f"Fund '{fund_name}' updated successfully.")
    else:
        print(f"Error: Fund '{fund_name}' not found.")

# List all funds in the SQLite database
def list_funds():
    cursor.execute("SELECT * FROM funds")
    funds = cursor.fetchall()
    if funds:
        print("Available Funds:")
        for fund in funds:
            print(f"ID: {fund[0]}, Name: {fund[1]}, Performance: {fund[2]:.2f}%")
    else:
        print("No funds available.")


# Close the database connection properly
def close_connection():
    connection.close()
    print("Database connection closed.")


# Code testing and validation
if __name__ == "__main__":
    initialize_database()

    while True:
        print("\nMenu:")
        print("1. New fund creation")
        print("2. Update Fund's Perfromance")
        print("3. Fund list")
        print("4. Exit")

        choice = input("Select an option (1-4): ")

        if choice == '1':
            fund_name = input("Enter fund name: ")
            performance = float(input("Enter fund performance (%): "))
            create_fund(fund_name, performance)
        elif choice == '2':
            fund_name = input("Enter fund name to update: ")
            new_performance = float(input("Enter new performance (%): "))
            update_fund(fund_name, new_performance)
        elif choice == '3':
            list_funds()
        elif choice == '4':
            close_connection()
            break
        else:
            print("Invalid option. Please try again.")



Menu:
1. New fund creation
2. Update Fund's Perfromance
3. Fund list
4. Exit


Select an option (1-4):  4


Database connection closed.


### Answer 2: With Tkinter GUI

In [2]:
import sqlite3
from tkinter import Tk, Label, Entry, Button, messagebox, END, Listbox

# Database setup
DB_FILE = 'funds.db'
connection = sqlite3.connect(DB_FILE)
cursor = connection.cursor()

# Initialize the funds database schema
def initialize_database():
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS funds (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        fund_name TEXT NOT NULL UNIQUE,
        performance REAL NOT NULL
    )
    """)
    connection.commit()

# Functions for GUI
# Add a new fund to the database
def add_fund():
    fund_name = entry_fund_name.get()
    try:
        performance = float(entry_performance.get())
        cursor.execute("INSERT INTO funds (fund_name, performance) VALUES (?, ?)", (fund_name, performance))
        connection.commit()
        messagebox.showinfo("Success", f"Fund '{fund_name}' added successfully.")
        entry_fund_name.delete(0, END)
        entry_performance.delete(0, END)
        list_funds()
    except sqlite3.IntegrityError:
        messagebox.showerror("Error", f"Fund '{fund_name}' already exists.")
    except ValueError:
        messagebox.showerror("Error", "Performance must be a valid number.")

# Update the performance of an existing fund
def update_fund():
    fund_name = entry_fund_name.get()
    try:
        new_performance = float(entry_performance.get())
        cursor.execute("UPDATE funds SET performance = ? WHERE fund_name = ?", (new_performance, fund_name))
        if cursor.rowcount > 0:
            connection.commit()
            messagebox.showinfo("Success", f"Fund '{fund_name}' updated successfully.")
            entry_fund_name.delete(0, END)
            entry_performance.delete(0, END)
            list_funds()
        else:
            messagebox.showerror("Error", f"Fund '{fund_name}' not found.")
    except ValueError:
        messagebox.showerror("Error", "Performance must be a valid number.")

# Display all funds in the listbox.
def list_funds():
    listbox_funds.delete(0, END)
    cursor.execute("SELECT * FROM funds")
    funds = cursor.fetchall()
    if funds:
        for fund in funds:
            listbox_funds.insert(END, f"ID: {fund[0]}, Name: {fund[1]}, Performance: {fund[2]:.2f}%")
    else:
        listbox_funds.insert(END, "No funds available.")

# Clear the entry fields
def clear_fields():
    entry_fund_name.delete(0, END)
    entry_performance.delete(0, END)

# GUI setup
root = Tk()
root.title("Fund Manager")

# Labels and input fields
Label(root, text="Fund Name:").grid(row=0, column=0, padx=10, pady=5)
entry_fund_name = Entry(root, width=30)
entry_fund_name.grid(row=0, column=1, padx=10, pady=5)

Label(root, text="Performance (%):").grid(row=1, column=0, padx=10, pady=5)
entry_performance = Entry(root, width=30)
entry_performance.grid(row=1, column=1, padx=10, pady=5)

# Buttons
Button(root, text="Add Fund", command=add_fund).grid(row=2, column=0, padx=10, pady=10)
Button(root, text="Update Fund", command=update_fund).grid(row=2, column=1, padx=10, pady=10)
Button(root, text="Clear Fields", command=clear_fields).grid(row=3, column=0, padx=10, pady=10)
Button(root, text="List Funds", command=list_funds).grid(row=3, column=1, padx=10, pady=10)

# Listbox to display funds
listbox_funds = Listbox(root, width=50, height=10)
listbox_funds.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

# Initialize the database and start the GUI event loop
initialize_database()
list_funds()
root.mainloop()
